# Vorausschauende Instandhaltung
## Geschäftskontext
Im diesem Beispiel analysieren wir einen maschinellen Klassifizierungsdatensatz für vorausschauende Wartung.
Da reale Predictive-Maintenance-Datensätze im Allgemeinen schwer zu beschaffen und vor allem schwer zu veröffentlichen sind, stellen wir einen synthetischen Datensatz vor, der nach unserem besten Wissen die reale Predictive Maintenance in der Industrie widerspiegelt.

Der Datensatz besteht aus 10 000 Datenpunkten, die als Zeilen mit 14 Merkmalen in Spalten gespeichert sind

- UID: eindeutiger Bezeichner im Bereich von 1 bis 10000
- ProductID: bestehend aus den Buchstaben L, M oder H für niedrige (50 % aller Produkte), mittlere (30 %) und hohe (20 %) Produktqualitätsvarianten und einer variantenspezifischen Seriennummer.
- Air temperature [K]: Die Lufttemperatur ist erzeugt mit einem Random-Walk-Prozess, der später auf eine Standardabweichung von 2 K um 300 K normalisiert wird
- Process temperature [K]: Die Prozesstemperatur erzeugt durch einen Random-Walk-Prozess, normiert auf eine Standardabweichung von 1 K, addiert zur Lufttemperatur plus 10 K.
- Rotational speed [U/min]: Die Drehzahl ist berechnet aus einer Leistung von 2860 W, überlagert mit einem normalverteilten Rauschen
- Torque [Nm]: Die Drehmomentwerte sind normalverteilt um 40 Nm mit einem Ïƒ = 10 Nm und keinen negativen Werten.
- Tool wear [min]: Die Qualitätsvarianten H/M/L fügen dem eingesetzten Werkzeug im Prozess 5/3/2 Minuten Werkzeugverschleiß hinzu.
- Machine failure: Zeigt an, ob die Maschine in diesem bestimmten Datenpunkt für eine der folgenden Ausfallarten ausgefallen ist.

Wichtig: Es gibt zwei Zielvariablen - machen Sie nicht den Fehler, eine von ihnen als Merkmal zu verwenden, da dies zu einem Leakage führen würde:
- Target: Versagen oder nicht
- Failure Type: Art des Versagens

In [2]:
import os
import re
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import plotly.express as px
import cv2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
# from torchviz import make_dot
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from PIL import Image

import holoviews as hv
from holoviews import opts
#hv.extension('bokeh')
import json
import shap

from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm


import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

from scipy.stats import norm, skew #for some statistics
from scipy import stats #qqplot
import statsmodels.api as sm #for decomposing the trends, seasonality etc.

from statsmodels.tsa.statespace.sarimax import SARIMAX #the big daddy

In [4]:
df = pd.read_csv('./data/predictive_maintenance.csv')
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14  Location                 10